# Projeto 2 - Backtest modelo de investimento em dividendos

<br> 

### Desafio:

* Testar se escolher empresas de dividendos funcionou no Brasil nos últimos anos.

<br> 

### Passo a passo:

   **Passo 1** - Importar os módulos e bibliotecas.
   
   **Passo 2** - Baixar os dados disponibilizados.
   
   **Passo 3** - Pegar as datas que iremos balancear a carteira. 
   
   **Passo 4** - Filtrar  a liquidez.
   
   **Passo 5** - Criar o ranking dos indicadores.
   
   **Passo 6** - Criar as carteiras. 
   
   **Passo 7** - Calcular a rentabilidade por carteira.
   
   **Passo 8** - Calcular a rentabilidade do modelo.

   **Passo 9** - Calcular a rentabilidade do ibovespa no mesmo período.

   **Passo 10** - Analisar os resultados.
   
   **Passo 11** - Comprar a carteira.
   
-------------------

# Passo 1: Importando os módulos necessários

In [ ]:
#!pip install quantstats

In [1]:
import pandas as pd
import quantstats as qs 

# Passo 2: Baixar os dados dispobinilizados. 

In [4]:
dados_empresas = pd.read_csv("datasets/dados_empresas.csv")
dados_empresas

,data,ticker,preco_fechamento_ajustado,volume_negociado,dy
0,2019-04-30,AALR3,14.87,2569789.0,0.005640
1,2019-05-31,AALR3,13.90,1586619.0,0.006030
2,2019-06-30,AALR3,14.30,10832812.0,0.005863
3,2019-07-31,AALR3,14.67,2810704.0,0.005716
4,2019-08-31,AALR3,18.99,4488459.0,0.004415
...,...,...,...,...,...
47302,2023-07-31,ZAMP3,4.42,4634150.0,0.000000
47303,2023-08-31,ZAMP3,5.55,14211395.0,0.000000
47304,2023-09-30,ZAMP3,6.13,8921464.0,0.000000
47305,2023-10-31,ZAMP3,6.35,6714596.0,0.000000


# Passo 3: Calcular os retornos mensais das empresas.

In [ ]:
dados_empresas['retorno'] = dados_empresas.groupby('ticker')['preco_fechamento_ajustado'].pct_change()
dados_empresas['retorno'] = dados_empresas.groupby('ticker')['retorno'].shift(-1) 

# Passo 4: Filtrar liquidez.

In [ ]:
dados_empresas = dados_empresas[dados_empresas['volume_negociado'] > 1000000]

# Passo 5: Criar o ranking dos indicadores.

In [ ]:
dados_empresas['ranking_dy'] = dados_empresas.groupby('data')['dy'].rank(ascending = False)

In [ ]:
#exemplo de carteira

dados_empresas[dados_empresas['data'] == '2017-01-31'].sort_values('ranking_dy').head(20)

# Passo 6: Criar as carteiras. 

In [ ]:
dados_empresas = dados_empresas[dados_empresas['ranking_dy'] <= 10]

#dados_empresas[dados_empresas['data'] == '2016-01-31']

# Passo 7: Calcula a rentabilidade por carteira.

In [ ]:
rentabilidade_por_carteiras = dados_empresas.groupby('data')['retorno'].mean()
rentabilidade_por_carteiras = rentabilidade_por_carteiras.to_frame()

# Passo 8: Calcula a rentabilidade do modelo.

In [ ]:
rentabilidade_por_carteiras['estrategia_dy'] = (rentabilidade_por_carteiras['retorno'] + 1).cumprod() - 1

rentabilidade_por_carteiras = rentabilidade_por_carteiras.shift(1)

rentabilidade_por_carteiras = rentabilidade_por_carteiras.dropna()

In [ ]:
rentabilidade_por_carteiras

# Passo 9: Calcula a rentabilidade do ibovespa no mesmo período.

In [ ]:
ibov = pd.read_csv('ibov.csv')

retornos_ibov = ibov['fechamento'].pct_change().dropna()
retorno_acum_ibov = (1 + retornos_ibov).cumprod() - 1
rentabilidade_por_carteiras['Ibovespa'] = retorno_acum_ibov.values
rentabilidade_por_carteiras = rentabilidade_por_carteiras.drop('retorno', axis = 1)

# Passo 10: Resultados.

In [ ]:
qs.extend_pandas()
rentabilidade_por_carteiras.index = pd.to_datetime(rentabilidade_por_carteiras.index)

rentabilidade_por_carteiras['estrategia_dy'].plot_monthly_heatmap()
rentabilidade_por_carteiras['Ibovespa'].plot_monthly_heatmap()

In [ ]:
rentabilidade_por_carteiras.plot(figsize = (7, 4))

# Passo 11: Comprar a última carteira gerada pelo modelo.

https://www.mql5.com/en/docs/python_metatrader5

In [ ]:
ultima_carteira = dados_empresas[dados_empresas['data'] == '2023-11-30']

tickers = ultima_carteira['ticker'].to_list()

In [ ]:
tickers

In [ ]:
import MetaTrader5 as mt5

In [ ]:
mt5.initialize()
for ticker in tickers:

    print(ticker)
    
    mt5.symbol_select(ticker)
    preco = mt5.symbol_info_tick(ticker).ask
    quantidade = 100.0
    ordem_compra = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": ticker,
        "volume": quantidade,
        "type": mt5.ORDER_TYPE_BUY,
        "price": preco,
        "magic": 1,
        "comment": "Trades automáticos",
        "type_time": mt5.ORDER_TIME_DAY,
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }
    result_compra = mt5.order_send(ordem_compra)